Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [82]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
from dotenv import load_dotenv

In [98]:
## configruation

load_dotenv('config.env')
OPEN_API_KEY = os.getenv('OPEN_API_KEY')

# List of JSON files
json_files = [('income_stmt', 'df_income_stmt_columns.json'), ('cashflow', 'df_cashflow_columns.json'), ('stocks', 'df_stocks_columns.json')]

# List to store loaded schemas
schemas = {}

# Load schemas from JSON files
for file in json_files:
    with open(file[1], 'r') as f:
        schema = json.load(f)
        schemas[file[0]] = schema

# Initialize an empty list to store the stocks
stocks_list = []

# Open the text file in read mode
with open('stocks.txt', 'r') as file:
    # Read each line of the file
    for line in file:
        # Strip any leading or trailing whitespace and append the line to the list
        stocks_list.append(line.strip())

df_portfolio = pd.read_csv('portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

client = OpenAI(api_key=OPEN_API_KEY)

In [102]:
# Upload a file with an "assistants" purpose
file_stocks = client.files.create(
  file=open("df_stocks.csv", "rb"),
  purpose='assistants'
)

file_income_stmt = client.files.create(
  file=open("df_income_stmt.csv", "rb"),
  purpose='assistants'
)

file_cashflow = client.files.create(
  file=open("df_cashflow.csv", "rb"),
  purpose='assistants'
)

In [127]:
client.files.delete(file_id=file_income_stmt_test.id)

file_income_stmt_test = client.files.create(
  file=open("df_income_stmt_test.csv", "rb"),
  purpose='assistants'
)

In [128]:
client.beta.assistants.update(assistant_id=fin_analyst.id, file_ids=[file_income_stmt_test.id])
# file_stocks.id

BadRequestError: Error code: 400 - {'error': {'message': 'Failed to index file: Unsupported file file-LQz9JLlp5HZnGCGwGjRJAF4h type: text/csv error_code: unhandled_mimetype', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [40]:
# I think financial analyst is too broad
# Need to divide the work. Maybe one RAG studies the financial statements
# One model evaluates the risk based on trends and sentiments

fin_analyst = client.beta.assistants.create(
    name='Financial Assistant',
    instructions='You are a financial assistant. Your task is to analyze the stock market and provide an analysis on the market or performance of specific stocks.\
         The objective is to identify whether the specific stocks would make a good investment for a 6 months period.',
    model='gpt-4-turbo-preview',
    tools=[{'type': 'retrieval'}]
)

In [41]:
## thread
thread = client.beta.threads.create()

In [42]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hi, can you explain to me what you do?"
)

In [43]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=fin_analyst.id
)

In [44]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [45]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [69]:
messages.data

[ThreadMessage(id='msg_YdTMX4NZiuei5NkAtzEy67pC', assistant_id='asst_I4lFSi5mtU5OYoeivxOc4piG', content=[MessageContentText(text=Text(annotations=[], value="Certainly! I'm a digital assistant designed to provide financial analysis, particularly regarding the stock market and investment opportunities. Here's a breakdown of what I can do for you:\n\n1. **Stock Market Analysis**: I can analyze stock market trends, including specific stocks or sectors. This includes looking at historical performance, current market conditions, recent news, and other relevant data to provide insights into potential future movements.\n\n2. **Investment Recommendations**: Based on an analysis of specific stocks or broader market conditions, I can offer insights into whether a particular stock or set of stocks might be a good investment over a specific time period, such as the next 6 months. This includes considering factors like financial health, growth potential, market volatility, and industry trends.\n\n3.

In [70]:
stocks_list = ['MSFT', 'SHOP', 'AMD', 'NVDA', 'AAPL', 'OKTA', 'SHOP', 'AMAT', 'TTWO', 'SNOW', 'SOXX', 'BA', 'DOCN', 'MCD', 'LULU', 'CSCO', 'ORCL', 'AMZN', 'ASML']

stocks_str = ' '.join(stocks_list)

stocks = yf.Tickers(stocks_str)

In [100]:
# How to set period parameter:
# week = wk
# month = mo
# year = y

# Create an empty df_stocks dataframe
df_stocks = pd.DataFrame(columns=dict(schemas['stocks']['columns']))

# Create an empty df_cashflow dataframe
df_cashflow = pd.DataFrame(columns=dict(schemas['cashflow']['columns']))

# Create an empty df_income_stmt dataframe
df_income_stmt = pd.DataFrame(columns=dict(schemas['income_stmt']['columns']))

# Loop through each stock
for stock in stocks_list:

    # Append to df_stocks
    df_plc = stocks.tickers[stock].history(period="max").reset_index()
    df_plc['Ticker'] = stock

    df_stocks = pd.concat([df_stocks, df_plc])

    # Append to df_cashflow
    df_plc = stocks.tickers[stock].cashflow.T.reset_index()
    df_plc.rename(columns={'index': 'Date'},inplace=True)
    df_plc['Ticker'] = stock

    df_cashflow = pd.concat([df_cashflow, df_plc])

    # Append to df_income_stmt
    df_plc = stocks.tickers[stock].income_stmt.T.reset_index()
    df_plc.rename(columns={'index': 'Date'},inplace=True)
    df_plc['Ticker'] = stock

    df_income_stmt = pd.concat([df_income_stmt, df_plc])

# Write to CSV
df_stocks.to_csv('df_stocks.csv', index=False)
df_cashflow.to_csv('df_cashflow.csv', index=False)
df_income_stmt.to_csv('df_income_stmt.csv', index=False)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10712\761198775.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_stocks = pd.concat([df_stocks, df_plc])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10712\761198775.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the ol

In [124]:
df_stocks.to_csv('df_stocks.csv', index=False)
df_cashflow.to_csv('df_cashflow.csv', index=False)
df_income_stmt.to_csv('df_income_stmt.csv', index=False)

In [112]:
# get historical market data
# hist = msft.history(period="1mo")
stocks.tickers['SHOP'].income_stmt.T.reset_index().dtypes
# df = stocks.tickers['SHOP'].cashflow.T.reset_index()
# df.rename(columns={'index': 'Date'},inplace=True)
# df

index                                                         datetime64[ns]
Tax Effect Of Unusual Items                                           object
Tax Rate For Calcs                                                    object
Normalized EBITDA                                                     object
Total Unusual Items                                                   object
Total Unusual Items Excluding Goodwill                                object
Net Income From Continuing Operation Net Minority Interest            object
Reconciled Depreciation                                               object
Reconciled Cost Of Revenue                                            object
EBITDA                                                                object
EBIT                                                                  object
Net Interest Income                                                   object
Interest Expense                                                      object